# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Dependencias necesarias

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional, Dict, List
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
from pymongo import MongoClient
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq



## 3. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [2]:
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


LLM de Groq inicializado correctamente
   Modelo: openai/gpt-oss-120b
   Temperature: 0.1


## 4. Conexión a Mongo DB

Cargamos la `MONGO_URI` desde el archivo `.env` e inicializamos.
IMPORTANTE: Contar con un usuario creado en la base de datos de Mongo. 

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
MONGO_URI=tu_clave_aqui
```

In [3]:
# Conectar a MongoDB Atlas
MONGO_URI = os.getenv("MONGO_URI")

if not MONGO_URI:
    raise ValueError("MONGO_URI no encontrada en .env")

client = MongoClient(MONGO_URI)
db = client.db_respuestos
collection = db.repuestos

print("✅ Conexión a MongoDB establecida")
print(f"   Base de datos: db_respuestos")
print(f"   Colección: repuestos")

✅ Conexión a MongoDB establecida
   Base de datos: db_respuestos
   Colección: repuestos


### ⚠️ Configuración del Índice Vectorial en MongoDB Atlas

**IMPORTANTE**: Para mejorar el rendimiento, debes agregar `proveedor_tipo` como campo filtrable en tu índice vectorial.

#### 📋 Configuración Actual (Temporal)
- ✅ Funcional: Los filtros se aplican **después** de la búsqueda vectorial
- ⚠️ Rendimiento: Puede ser más lento para datasets grandes

#### 🚀 Configuración Óptima (Recomendada)

1. **Ve a MongoDB Atlas** → Tu Cluster → Browse Collections
2. **Selecciona tu colección** de repuestos
3. **Haz clic en "Search Indexes"**
4. **Edita tu índice** `vector_index_repuestos`
5. **Agrega el campo filtrable** en la configuración JSON:

```json
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding_vector",
      "numDimensions": 384,
      "similarity": "cosine"
    },
    {
      "type": "filter",
      "path": "proveedor_tipo"
    },
    {
      "type": "filter",
      "path": "stock_disponible"
    }
  ]
}
```

6. **Guarda** y espera que el índice se reconstruya (~2-5 minutos)

#### 🔄 Después de actualizar el índice

Puedes **revertir** el código para usar filtros en el pipeline (más eficiente):

```python
# En semantic_search_internal y semantic_search_external
pipeline = [
    {
        "$vectorSearch": {
            # ...
            "filter": {
                "proveedor_tipo": {"$eq": "INTERNAL"}  # o "EXTERNAL"
            }
        }
    }
]
```

Y eliminar la línea de filtrado post-búsqueda:
```python
# REMOVER ESTA LÍNEA:
resultados = [r for r in resultados_raw if r.get('proveedor_tipo') == 'INTERNAL']
```


## 5. Definicion del Estado 

In [4]:
class ValidationRequest(BaseModel):
    is_parts_request: bool = Field(
        default=False,
        description="True si la consulta es una solicitud de repuestos o piezas. False si es una pregunta general o spam."
    )
    message: str = Field(
        default="",
        description="Mensaje del agente. Si es un pedido de repuestos, debe incluir los pasos siguientes. Indicar al cliente que debe se deben realizar consultas sobre repuestos."
    )

class ConversationResult(BaseModel):
    """
    Resultado del análisis conversacional.
    """
    enough_info: bool = Field(
        description="True si hay suficiente información para proceder con la búsqueda, False si se necesita más información"
    )
    message: str = Field(
        description="Mensaje para el usuario (pregunta si enough_info=False, confirmación si enough_info=True)"
    )

class ProductList(BaseModel):
    products: list[str] = Field(
        default=[],
        description="Es una lista con las descripciones que realizo el cliente de los productos que solicita"
    )

class Repuesto(BaseModel):
    """
    Representa un repuesto identificado en la búsqueda semántica.
    Incluye código, descripción, marca, modelo y score de similitud.
    """
    id_repuesto: str = Field(
        description="Código único del repuesto (ej: R-0001)"
    )
    repuesto_descripcion: str = Field(
        description="Descripción del repuesto"
    )
    marca: str = Field(
        default="N/A",
        description="Marca del repuesto (SKF, FAG, Bosch, etc.)"
    )
    modelo: str = Field(
        default="N/A",
        description="Modelo o número de parte del repuesto"
    )
    categoria: str = Field(
        default="N/A",
        description="Categoría del repuesto (RODAMIENTO, FILTRO, etc.)"
    )
    score: float = Field(
        default=0.0,
        description="Score de similitud de la búsqueda semántica (0.0 a 1.0)"
    )
    
    def __str__(self) -> str:
        """Representación legible del repuesto"""
        return f"{self.id_repuesto} ({self.marca}): {self.repuesto_descripcion}"
    
    def get_unique_key(self) -> str:
        """Genera una clave única para identificar esta variante específica"""
        return f"{self.id_repuesto}_{self.marca}_{self.modelo}"

class ProductInfoVerification(BaseModel):
    """
    Resultado de la verificación de información de un producto.
    """
    info_completa: bool = Field(
        description="True si hay suficiente información para buscar (al menos tipo de repuesto identificado)"
    )
    razon: str = Field(
        description="Explicación breve de por qué sí o no tiene información suficiente"
    )
    info_faltante: List[str] = Field(
        default=[],
        description="Lista de campos que faltan o necesitan más detalle"
    )

class UserSelectionIntent(BaseModel):
    """
    Interpretación de la intención del usuario al seleccionar productos.
    """
    accion: str = Field(
        description="La acción que el usuario quiere realizar: 'confirmar_todo', 'seleccionar_codigos', 'cancelar', 'no_entendido'"
    )
    codigos_seleccionados: List[str] = Field(
        default=[],
        description="Lista de códigos de productos mencionados (formato R-XXXX). Vacío si confirmó todo o canceló."
    )
    confianza: float = Field(
        description="Nivel de confianza en la interpretación (0.0 a 1.0)"
    )
    razon: str = Field(
        description="Breve explicación de por qué se interpretó así"
    )

#Definimos el esquema mejorado
class AgentState(TypedDict):
    validation_result: ValidationRequest
    conversation_result: Optional[ConversationResult]  # Resultado del análisis conversacional
    messages: Annotated[list, add_messages]
    product_description: List[str]
    product_requests: Optional[List[Dict]]  # Lista de productos con tracking de info completa
    codigos_repuestos: Optional[List[str]]  # Lista de códigos únicos (R-XXXX)
    repuestos_encontrados: Optional[List[Repuesto]]  # Lista de objetos Repuesto con todas las variantes
    productos_sin_match_interno: Optional[List[Dict]]  # Productos que no se encontraron internamente
    info_completa: bool  # Si tenemos toda la información necesaria

    # Query optimizada para búsqueda semántica
    optimized_query: Optional[str]  # Query reformulada por el LLM

    # Resultados de búsqueda semántica
    semantic_results: Optional[List[Dict]]  # Candidatos de búsqueda semántica

    # Resultados de la búsqueda interna
    resultados_internos: Optional[Dict[str, List[Dict]]]  

    # Resultados de la búsqueda externa
    resultados_externos: Optional[Dict[str, List[Dict]]]

    # Análisis de disponibilidad por código
    disponibilidad: Optional[Dict[str, str]]  # {"R-0001": "full", "R-0002": "none"}
    codigos_para_externos: Optional[List[str]]

    # Reranking
    recomendaciones_llm: Optional[str]
    
    # Human in the loop
    selecciones_usuario: Optional[List[Dict]]  # Selecciones del usuario después del ranking
    repuestos_seleccionados: bool  # True cuando el usuario ha seleccionado productos válidos

## 6. Definicion de todas las Chains del LLM

### Chain de Extracción de respuestos

In [5]:
with open('prompts/identify_products.txt', 'r', encoding='utf-8') as f:
    IDENTIFY_PRODUCTS_PROMPT = f.read()

prompt = ChatPromptTemplate.from_messages([
    ("system", IDENTIFY_PRODUCTS_PROMPT),
    ("placeholder", "{messages}")
])

# Chain conversacional con respuesta estructurada
extraction_chain = prompt | llm.with_structured_output(ProductList)

### Chain de Conversación Inicial

In [6]:
with open('prompts/chat_inicial_prompt.txt', 'r', encoding='utf-8') as f:
    SYSTEM_PROMPT = f.read()

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

# Chain conversacional con respuesta estructurada
conversational_chain = prompt | llm.with_structured_output(ConversationResult)

### Chain de Validación de Intencion

In [7]:
with open('prompts/intention_classifier_prompt.txt', 'r', encoding='utf-8') as f:
    CLASSIFIER_SYSTEM_PROMPT = f.read()

validation_prompt = ChatPromptTemplate.from_messages([
    ("system", CLASSIFIER_SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

parser = PydanticOutputParser(pydantic_object=ValidationRequest)

validation_prompt_con_instrucciones = ChatPromptTemplate.from_messages(
    validation_prompt.messages + [
        ("human", "{format_instructions}")
    ]
).partial(
    format_instructions=parser.get_format_instructions()
)

validation_chain = (
    validation_prompt_con_instrucciones 
    | llm 
    | parser
)

### Chain de Reranking

In [8]:
with open('prompts/reranking_prompt.txt', 'r', encoding='utf-8') as f:
    RERANKING_PROMPT = f.read()

ranking_prompt_template = ChatPromptTemplate.from_messages([
    ("system", RERANKING_PROMPT),
    ("user", "Analiza las siguientes opciones y genera un ranking completo:\n\n{opciones_texto}")
])

ranking_chain = ranking_prompt_template | llm

### Chain de Requery Optimization


In [9]:
with open('prompts/requery_optimization_prompt.txt', 'r', encoding='utf-8') as f:
    REQUERY_PROMPT = f.read()

requery_prompt_template = ChatPromptTemplate.from_messages([
    ("system", REQUERY_PROMPT),
    ("user", "{user_message}")
])

requery_chain = requery_prompt_template | llm

### Chain de Verificación de Información de Productos


In [10]:
VERIFY_PRODUCT_INFO_PROMPT = """
Eres un experto en repuestos industriales. Analiza si la siguiente descripción de producto tiene información SUFICIENTE para realizar una búsqueda precisa en un catálogo.

Descripción del producto: "{product_name}"

Información necesaria para búsqueda efectiva:
- Tipo de repuesto (rodamiento, filtro, bomba, correa, etc.) - REQUERIDO
- Marca (SKF, Bosch, FAG, Parker, etc.) - OPCIONAL pero recomendado
- Modelo o número de parte (6204-2RS, HF35554, etc.) - OPCIONAL pero recomendado

Si tiene al menos el TIPO DE REPUESTO claramente identificado, la información es suficiente para intentar una búsqueda por similitud semántica.

Ejemplos:
- "rodamiento" → info_completa: true (tipo claro)
- "filtro Bosch" → info_completa: true (tipo + marca)
- "repuesto" → info_completa: false (muy vago, no se identifica tipo)
- "algo para motor" → info_completa: false (no hay tipo específico)
- "rodamiento SKF 6204-2RS" → info_completa: true (tipo + marca + modelo, ideal)

Responde en formato JSON estructurado.
"""

verify_info_prompt = ChatPromptTemplate.from_messages([
    ("system", VERIFY_PRODUCT_INFO_PROMPT),
])

verify_product_chain = verify_info_prompt | llm.with_structured_output(ProductInfoVerification)


### Chain de Interpretación de Selección de Usuario


In [11]:
INTERPRET_SELECTION_PROMPT = """
Eres un asistente experto en interpretar las respuestas de usuarios en un sistema de pedidos de repuestos.

El usuario está viendo un ranking de productos y debe seleccionar cuál(es) desea ordenar.

OPCIONES DISPONIBLES EN EL RANKING:
{codigos_disponibles}

RESPUESTA DEL USUARIO:
"{user_message}"

Tu tarea es interpretar qué quiere hacer el usuario:

1. **confirmar_todo**: Si dice que quiere todos los productos, confirma, acepta, procede con todo, etc.
   Ejemplos: "sí", "confirmar", "ok", "todos", "adelante", "proceder"

2. **seleccionar_codigos**: Si menciona códigos específicos (formato R-XXXX).
   Ejemplos: "R-0001", "quiero el R-0007", "dame R-0002 y R-0005"
   IMPORTANTE: Extrae SOLO los códigos que están en el formato R-XXXX

3. **cancelar**: Si rechaza, cancela, no quiere nada.
   Ejemplos: "no", "cancelar", "no gracias", "rechazar"

4. **no_entendido**: Si la respuesta es ambigua o no se puede interpretar claramente.

IMPORTANTE:
- Sé PERMISIVO: si el usuario menciona un código (R-XXXX), interpreta como selección aunque la frase no sea perfecta
- Extrae TODOS los códigos en formato R-XXXX que encuentres en el mensaje
- Si hay códigos, la acción es 'seleccionar_codigos' (no 'confirmar_todo')
- Usa alta confianza (>0.8) si la intención es clara

Responde en formato JSON estructurado.
"""

selection_prompt = ChatPromptTemplate.from_messages([
    ("system", INTERPRET_SELECTION_PROMPT),
])

selection_interpretation_chain = selection_prompt | llm.with_structured_output(UserSelectionIntent)


## 7. Definición de los Nodos del Grafo

In [12]:
def extract_products_info(state: AgentState) -> AgentState:
    """
    Usa la chain de extracción estructurada para identificar todos los productos
    descritos por el usuario en los mensajes.
    MERGEA con productos anteriores que ya tienen información completa.
    """
    messages: list[BaseMessage] = state['messages']
    existing_requests = state.get('product_requests', [])
        
    # Solo invocamos con el último mensaje o la colección de mensajes
    try:
        # Usamos la cadena de extracción que retorna ProductList
        product_list_obj = extraction_chain.invoke({"messages": messages})
        
        # Convertir a la nueva estructura de seguimiento
        new_product_requests = []
        for product_name in product_list_obj.products:
            new_product_requests.append({
                "name": product_name,
                "info_needed": True,  # Inicialmente se asume que se necesita más info
                "details": {},
                "info_solicitada": ["descripción detallada", "marca", "modelo/número de parte"]
            })
        
        # MERGEAR: mantener productos que ya estaban completos (info_needed=False)
        productos_previos_completos = [
            p for p in existing_requests 
            if not p.get("info_needed", True)
        ]
        
        if productos_previos_completos:
            for p in productos_previos_completos:
                print(f"      - {p.get('name', 'sin nombre')}")
        
        # Combinar: primero los completos previos, luego los nuevos
        product_requests = productos_previos_completos + new_product_requests
        
        return {
            "product_requests": product_requests,
            "product_description": product_list_obj.products  # Mantenemos por si se usa en otra parte
        }
    
    except Exception as e:
        # En caso de error, mantener productos existentes o volver a solicitar información
        return {
            "product_requests": existing_requests if existing_requests else [],
            "messages": [AIMessage(content="❌ Lo siento, no pude identificar los repuestos que necesitas. ¿Podrías ser más específico?")]
        }


In [13]:
def check_product_info_completeness(state: AgentState) -> AgentState:
    """
    Verifica si CADA producto en product_requests tiene información completa.
    Usa el LLM para analizar cada descripción individualmente.
    Retorna info_completa=True solo si TODOS los productos están completos.
    """
    product_requests = state.get('product_requests', [])
    
    if not product_requests:
        return {
            "info_completa": False,
            "messages": [AIMessage(content="No se identificaron productos para buscar.")]
        }
    
    productos_incompletos = []
    productos_completos = []
    
    # Verificar CADA producto con el LLM
    for idx, product in enumerate(product_requests, 1):
        product_name = product.get("name", "")
        
        try:
            # Invocar LLM para verificar este producto
            verificacion = verify_product_chain.invoke({"product_name": product_name})
            
            # Actualizar el product_request con el resultado
            product["info_needed"] = not verificacion.info_completa
            product["verificacion"] = {
                "completa": verificacion.info_completa,
                "razon": verificacion.razon,
                "faltante": verificacion.info_faltante
            }
            
            if verificacion.info_completa:
                productos_completos.append({
                    "idx": idx,
                    "nombre": product_name,
                    "razon": verificacion.razon
                })
                print(f"✅ Producto {idx}: '{product_name}'")
                print(f"   → Información completa: {verificacion.razon}\n")
            else:
                productos_incompletos.append({
                    "idx": idx,
                    "nombre": product_name,
                    "razon": verificacion.razon,
                    "faltante": verificacion.info_faltante
                })
                print(f"⚠️  Producto {idx}: '{product_name}'")
                print(f"   → Información incompleta: {verificacion.razon}")
                if verificacion.info_faltante:
                    print(f"   → Falta: {', '.join(verificacion.info_faltante)}\n")
                else:
                    print()
                
        except Exception as e:
            print(f"❌ Error verificando producto {idx}: {e}")
            # Por defecto, marcar como incompleto si hay error
            product["info_needed"] = True
            productos_incompletos.append({
                "idx": idx,
                "nombre": product_name,
                "razon": "Error al verificar",
                "faltante": ["detalles"]
            })
    
    # DECISIÓN: Si ALGÚN producto está incompleto → pedir más info
    if productos_incompletos:
        mensaje = "⚠️  Necesito más detalles sobre algunos productos:\n\n"
        
        for item in productos_incompletos:
            mensaje += f"**{item['idx']}. {item['nombre']}**\n"
            mensaje += f"   ❌ {item['razon']}\n"
            if item['faltante']:
                mensaje += f"   📝 Por favor especifica: {', '.join(item['faltante'])}\n"
            mensaje += "\n"
        
        mensaje += "💡 **Ejemplos de descripciones completas:**\n"
        mensaje += "   • 'Rodamiento rígido de bolas 6204'\n"
        mensaje += "   • 'Filtro de aceite Bosch'\n"
        mensaje += "   • 'Bomba centrífuga Parker'\n"
        mensaje += "   • 'Correa trapezoidal SKF'\n\n"
        mensaje += "Por favor, proporciona más detalles de los productos incompletos."
        
        return {
            "info_completa": False,
            "product_requests": product_requests,  # Actualizado con flags
            "messages": [AIMessage(content=mensaje)]
        }
    else:        
        mensaje = "✅ Perfecto, tengo información suficiente para buscar:\n\n"
        for item in productos_completos:
            mensaje += f"   {item['idx']}. {item['nombre']}\n"
        mensaje += "\n🔍 Iniciando búsqueda en el catálogo..."
        
        return {
            "info_completa": True,
            "product_requests": product_requests,
            "messages": [AIMessage(content=mensaje)]
        }

def route_after_extraction_check(state: AgentState) -> str:
    """
    Decide si continuar con semantic_search o pedir más info.
    """
    if state.get("info_completa", False):
        return "semantic_search"  # ← Todos completos, buscar
    else:
        return "request_more_info"  # ← Alguno incompleto, pedir más info


In [14]:
def classify_request(state: AgentState) -> AgentState:
    """
    Clasifica la solicitud del usuario.
    """
    messages = state['messages']
    validation_result_object = validation_chain.invoke({"messages": messages})
    
    return {"validation_result": validation_result_object}

def set_val_message(state: AgentState) -> AgentState:
    """
    Establece el mensaje de validación.
    """
    return {"messages": [state['validation_result'].message]}

def route_classification(state: AgentState) -> str:
    """
    Ruta de clasificación.
    """
    if state['validation_result'].is_parts_request:
        return "continue"
    else:
        return "end"

def route_after_semantic(state: AgentState) -> str:
    """
    Decide si continuar con búsqueda interna/externa o pedir más info.
    """
    info_completa = state.get("info_completa", False)
    codigos = state.get("codigos_repuestos")
    
    if info_completa and codigos and len(codigos) > 0:
        return "search_internal_parts"
    else:
        return "request_more_info"

def search_internal_parts(state: AgentState) -> AgentState:
    """
    Busca el repuesto en MongoDB usando el código identificado.
    Muestra los resultados por pantalla de forma detallada.
    """
    codigos = state.get("codigos_repuestos")
    
    try:
        # Buscar en MongoDB por id_repuesto
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos},
            "proveedor_tipo": "INTERNAL"
        }))

        resultados_por_codigo = {codigo: [] for codigo in codigos}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                # Limpiar _id de MongoDB para serialización
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
        
        # Mostrar resultados básicos
        for codigo in codigos:
            opciones = resultados_por_codigo[codigo]
        return {"resultados_internos": resultados_por_codigo}

    except Exception as e:
        print(f"❌ Error al buscar en la base de datos: {e}")
        return {"resultados_internos": {codigo: [] for codigo in codigos}}


def search_external_parts(state: AgentState) -> AgentState:
    """
    Busca en proveedores externos solo los repuestos que lo necesitan.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    try:
        # Búsqueda en batch
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos_para_externos},
            "proveedor_tipo": "EXTERNAL"
        }))
        
        # Organizar por código
        resultados_por_codigo = {codigo: [] for codigo in state.get("codigos_repuestos", [])}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
        
        for codigo in codigos_para_externos:
            count = len(resultados_por_codigo.get(codigo, []))
        
        
        return {"resultados_externos": resultados_por_codigo}
        
    except Exception as e:
        print(f"❌ Error al buscar en proveedores externos: {str(e)}\n")
        return {"resultados_externos": {}}

def need_external_parts(state: AgentState) -> str:
    """
    Decide si se necesita buscar en proveedores externos.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    
    if codigos_para_externos:
        return "search_external"
    else:
        return "reranking"

def generate_ranking(state: AgentState) -> AgentState:
    """
    Genera ranking usando SOLO el LLM.
    """
    from utils import format_options_for_llm
    
    codigos = state.get("codigos_repuestos", [])
    resultados_internos = state.get("resultados_internos", {})
    resultados_externos = state.get("resultados_externos", {})
    
    # Preparar información para el LLM
    opciones_para_llm = []
    
    for codigo in codigos:
        internos = resultados_internos.get(codigo, [])
        externos = resultados_externos.get(codigo, [])
        
        # Combinar todas las opciones
        todas_opciones = []
        
        for opcion in internos:
            todas_opciones.append({**opcion, "tipo": "INTERNO"})
        for opcion in externos:
            todas_opciones.append({**opcion, "tipo": "EXTERNO"})
            
        if todas_opciones:
            # Formatear para el LLM
            opciones_para_llm.append(
                format_options_for_llm(codigo, todas_opciones)
            )
    
    if not opciones_para_llm:
        print("⚠️ No hay opciones para rankear\n")
        return {
            "recomendaciones_llm": "No hay opciones disponibles."
        }
    
    # Crear el texto completo para el LLM
    opciones_texto = "\n\n".join(opciones_para_llm)
    print(f"Estas son las opciones!! {opciones_texto}")
    
    try:
        # Invocar el LLM para que haga el ranking
        recomendaciones = ranking_chain.invoke({"opciones_texto": opciones_texto})
        recomendaciones_texto = recomendaciones.content
        
    except Exception as e:
        print(f"❌ Error al obtener ranking del LLM: {e}\n")
        recomendaciones_texto = "Error al generar ranking automático."
    
    return {
        "recomendaciones_llm": recomendaciones_texto
    }


### 👤 Human in the Loop (HITL)

**Flujo de confirmación del usuario:**

1. **Reranking** → El LLM genera el ranking de mejores opciones
2. **human_in_the_loop** → Se presenta el ranking y se solicita confirmación
3. **INTERRUPT** ⏸️ → El grafo se pausa esperando respuesta del usuario
4. **process_selection** → Se procesa la selección del usuario
5. **Finalización** → Se procede según la decisión del usuario

**Opciones del usuario:**
- `'sí'` / `'confirmar'` → Confirma todas las mejores opciones
- `'no'` / `'cancelar'` → Cancela el pedido
- `'R-0001, R-0005'` → Selecciona códigos específicos

**Implementación técnica:**
- `interrupt_before=["process_selection"]` en `graph.compile()`
- `MemorySaver` para mantener el estado durante la pausa
- `selecciones_usuario` en `AgentState` para guardar la elección


In [15]:
def human_in_the_loop_selection(state: AgentState) -> AgentState:
    """
    Presenta el ranking al usuario y solicita su selección.
    Este nodo se ejecuta DESPUÉS del reranking.
    """
    recomendaciones = state.get("recomendaciones_llm", "")
    codigos = state.get("codigos_repuestos", [])
    
    # Mostrar el ranking (ya fue mostrado por el nodo anterior)
    mensaje = "📋 **Resumen del ranking:**\n\n"
    mensaje += recomendaciones + "\n\n"
    mensaje += "─" * 60 + "\n\n"
    mensaje += "🤔 **¿Deseas proceder con alguna de estas opciones?**\n\n"
    mensaje += "Por favor indica:\n"
    mensaje += "• **'sí'** o **'confirmar'** - Para proceder con las mejores opciones\n"
    mensaje += "• **'no'** o **'cancelar'** - Para cancelar el pedido\n"
    mensaje += "• **[código(s)]** - Para seleccionar específicos (ej: 'R-0001, R-0005')\n"
    
    return {
        "messages": [AIMessage(content=mensaje)]
    }

def process_user_selection(state: AgentState) -> AgentState:
    """
    Procesa la selección del usuario usando LLM para interpretar su intención.
    Valida que los códigos seleccionados estén en el ranking.
    Determina si son productos internos o externos.
    """
    messages = state.get("messages", [])
    codigos_disponibles = state.get("codigos_repuestos", [])
    resultados_internos = state.get("resultados_internos", {})
    resultados_externos = state.get("resultados_externos", {})
    
    # Obtener el último mensaje del usuario
    user_message = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            user_message = msg.content.strip()
            break
    
    if not user_message:
        return {
            "messages": [AIMessage(content="❌ No recibí tu selección. ¿Podrías responder de nuevo?")],
            "repuestos_seleccionados": False
        }
    
    
    # Usar LLM para interpretar la intención del usuario
    try:
        interpretation = selection_interpretation_chain.invoke({
            "codigos_disponibles": ", ".join(codigos_disponibles),
            "user_message": user_message
        })
        
    except Exception as e:
        print(f"❌ Error al interpretar con LLM: {e}")
        # Fallback a regex simple
        import re
        codigos_mencionados = re.findall(r'R-\d{4}', user_message.upper())
        if codigos_mencionados:
            interpretation = UserSelectionIntent(
                accion="seleccionar_codigos",
                codigos_seleccionados=codigos_mencionados,
                confianza=0.7,
                razon="Fallback a regex"
            )
        else:
            interpretation = UserSelectionIntent(
                accion="no_entendido",
                codigos_seleccionados=[],
                confianza=0.3,
                razon="Error en interpretación"
            )
    
    # Procesar según la acción interpretada
    if interpretation.accion == "cancelar":
        print(f"❌ Usuario canceló el pedido")
        
        mensaje = "❌ **Pedido cancelado**\n\n"
        mensaje += "Entendido, no se procesará ningún pedido.\n"
        mensaje += "¿Hay algo más en lo que pueda ayudarte?"
        
        return {
            "messages": [AIMessage(content=mensaje)],
            "selecciones_usuario": [],
            "repuestos_seleccionados": True
        }
    
    elif interpretation.accion == "confirmar_todo":
        print(f"✅ Usuario confirmó todas las opciones")
        codigos_validos = codigos_disponibles
        
    elif interpretation.accion == "seleccionar_codigos":
        codigos_mencionados = interpretation.codigos_seleccionados
        print(f"📋 Códigos seleccionados: {codigos_mencionados}")
        
        # Validar que los códigos existan en el ranking
        codigos_invalidos = [c for c in codigos_mencionados if c not in codigos_disponibles]
        codigos_validos = [c for c in codigos_mencionados if c in codigos_disponibles]
        
        if codigos_invalidos:
            print(f"⚠️  Códigos inválidos: {codigos_invalidos}")
            
            mensaje = f"❌ **Códigos no válidos**\n\n"
            mensaje += f"Los siguientes códigos NO están en el ranking:\n"
            for codigo in codigos_invalidos:
                mensaje += f"   • **{codigo}**\n"
            
            mensaje += f"\n✅ **Opciones disponibles:**\n\n"
            for codigo in codigos_disponibles:
                info_interno = resultados_internos.get(codigo, [])
                info_externo = resultados_externos.get(codigo, [])
                
                if info_interno:
                    info = info_interno[0]
                    tipo = "INTERNO"
                elif info_externo:
                    info = info_externo[0]
                    tipo = "EXTERNO"
                else:
                    continue
                
                desc = info.get('repuesto_descripcion', 'N/A')
                proveedor = info.get('proveedor_nombre', 'N/A')
                mensaje += f"   • **{codigo}** ({tipo})\n"
                mensaje += f"      └─ {desc}\n"
                mensaje += f"      └─ {proveedor}\n\n"
            
            mensaje += "Por favor selecciona códigos válidos del ranking."
            return {
                "messages": [AIMessage(content=mensaje)],
                "repuestos_seleccionados": False
            }
        
        if not codigos_validos:
            mensaje = "❓ No detecté ningún código válido. Por favor especifica uno de los códigos del ranking."
            return {
                "messages": [AIMessage(content=mensaje)],
                "repuestos_seleccionados": False
            }
    
    else:  # no_entendido
        print(f"❓ No se pudo interpretar la respuesta")
        mensaje = "❓ **No entendí tu respuesta**\n\n"
        mensaje += "Por favor indica:\n"
        mensaje += "• **'confirmar'** - Para proceder con todas las opciones\n"
        mensaje += "• **'cancelar'** - Para no hacer pedido\n"
        mensaje += f"• **[código]** - Para seleccionar específicos\n\n"
        mensaje += "**Códigos disponibles:**\n"
        for codigo in codigos_disponibles:
            mensaje += f"   • {codigo}\n"
        
        return {
            "messages": [AIMessage(content=mensaje)],
            "repuestos_seleccionados": False
        }
    
    
    mensaje_final = "✅ **Pedido confirmado**\n\n"
    
    tiene_internos = False
    tiene_externos = False
    
    for codigo in codigos_validos:
        info_interno = resultados_internos.get(codigo, [])
        info_externo = resultados_externos.get(codigo, [])
        
        if info_interno:
            tipo = "INTERNO"
            info = info_interno[0]
            tiene_internos = True
        elif info_externo:
            tipo = "EXTERNO"
            info = info_externo[0]
            tiene_externos = True
        else:
            continue
        
        desc = info.get('repuesto_descripcion', 'N/A')
        marca = info.get('marca', 'N/A')
        proveedor = info.get('proveedor_nombre', 'N/A')
                
        mensaje_final += f"📦 **{codigo}** - {tipo}\n"
        mensaje_final += f"   └─ {desc}\n"
        mensaje_final += f"   └─ Marca: {marca}\n"
        mensaje_final += f"   └─ Proveedor: {proveedor}\n\n"
    
    # Determinar tipo de orden
    if tiene_internos and tiene_externos:
        tipo_orden = "both"
    elif tiene_internos:
        tipo_orden = "internal_only"
    else:
        tipo_orden = "external_only"
        
    mensaje_final += "─" * 60 + "\n"
    mensaje_final += "🔄 Procesando tu pedido...\n"
    
    return {
        "messages": [AIMessage(content=mensaje_final)],
        "selecciones_usuario": [{"codigo": c, "tipo": tipo_orden} for c in codigos_validos],
        "repuestos_seleccionados": True,
        "tipo_orden": tipo_orden
    }
def route_after_selection(state: AgentState) -> str:
    """
    Decide si proceder con el pedido o terminar.
    """
    selecciones = state.get("selecciones_usuario", [])
    
    if selecciones and len(selecciones) > 0:
        return "process_order"  # Procesar pedido
    else:
        return "end"  # Terminar (usuario canceló o no hay selección)


In [16]:
def process_order(state: AgentState) -> AgentState:
    """
    Procesa la orden de compra con los productos seleccionados.
    Genera un resumen detallado por tipo (INTERNO/EXTERNO) y proveedor.
    """
    selecciones = state.get("selecciones_usuario", [])
    
    if not selecciones:
        return {
            "messages": [AIMessage(content="❌ No hay productos seleccionados para procesar")]
        }
        
    # Separar por tipo
    internos = [s for s in selecciones if s['tipo'] == 'INTERNO']
    externos = [s for s in selecciones if s['tipo'] == 'EXTERNO']
    
    # Agrupar externos por proveedor
    externos_por_proveedor = {}
    for ext in externos:
        prov = ext.get('proveedor', 'Desconocido')
        if prov not in externos_por_proveedor:
            externos_por_proveedor[prov] = []
        externos_por_proveedor[prov].append(ext)
    
    mensaje = "═" * 80 + "\n"
    mensaje += "📋 **ORDEN DE COMPRA GENERADA**\n"
    mensaje += "═" * 80 + "\n\n"
    
    # PRODUCTOS INTERNOS
    if internos:
        mensaje += "🏢 **PRODUCTOS INTERNOS (Almacén)**\n"
        mensaje += "─" * 80 + "\n"
        
        total_interno = 0
        lead_time_max_interno = 0
        
        for i, prod in enumerate(internos, 1):
            precio = prod.get('precio', 0)
            stock = prod.get('stock', 'N/A')
            lead_time = prod.get('lead_time', 0)
            
            total_interno += precio
            if isinstance(lead_time, (int, float)):
                lead_time_max_interno = max(lead_time_max_interno, int(lead_time))
            
            mensaje += f"{i}. **{prod['codigo']}** - {prod['descripcion']}\n"
            mensaje += f"   • Almacén: {prod['proveedor']}\n"
            mensaje += f"   • Cantidad: 1\n"
            mensaje += f"   • Precio unitario: ${precio:.2f}\n"
            mensaje += f"   • Stock disponible: {stock}\n"
            mensaje += f"   • Subtotal: ${precio:.2f}\n\n"
        
        mensaje += f"📊 **Total INTERNO:** ${total_interno:.2f}\n"
        mensaje += f"⏱️  **Disponibilidad:** Inmediata ({lead_time_max_interno} día{'s' if lead_time_max_interno != 1 else ''})\n\n"
        mensaje += "─" * 80 + "\n\n"
    
    # PRODUCTOS EXTERNOS
    if externos:
        mensaje += "🌐 **PRODUCTOS EXTERNOS (Proveedores)**\n"
        mensaje += "─" * 80 + "\n"
        
        total_externo = 0
        lead_time_max_externo = 0
        
        for proveedor, productos in externos_por_proveedor.items():
            mensaje += f"**PROVEEDOR: {proveedor}**\n"
            mensaje += "─" * 80 + "\n"
            
            for i, prod in enumerate(productos, 1):
                precio = prod.get('precio', 0)
                lead_time = prod.get('lead_time', 0)
                
                total_externo += precio
                if isinstance(lead_time, (int, float)):
                    lead_time_max_externo = max(lead_time_max_externo, int(lead_time))
                
                mensaje += f"{i}. **{prod['codigo']}** - {prod['descripcion']}\n"
                mensaje += f"   • Cantidad: 1\n"
                mensaje += f"   • Precio unitario: ${precio:.2f}\n"
                mensaje += f"   • Subtotal: ${precio:.2f}\n"
                mensaje += f"   • Lead time: {lead_time} días\n\n"
            
        mensaje += f"📊 **Total EXTERNO:** ${total_externo:.2f}\n"
        mensaje += f"⏱️  **Lead time estimado:** {lead_time_max_externo} días\n\n"
        mensaje += "─" * 80 + "\n\n"
    
    # RESUMEN FINANCIERO
    total_general = sum(p.get('precio', 0) for p in selecciones)
    
    mensaje += "💰 **RESUMEN FINANCIERO**\n"
    mensaje += "─" * 80 + "\n"
    if internos:
        total_int = sum(p.get('precio', 0) for p in internos)
        mensaje += f"   Productos internos: ${total_int:.2f}\n"
    if externos:
        total_ext = sum(p.get('precio', 0) for p in externos)
        mensaje += f"   Productos externos: ${total_ext:.2f}\n"
    mensaje += "   " + "─" * 30 + "\n"
    mensaje += f"   **TOTAL ESTIMADO:**    **${total_general:.2f}**\n\n"
    
    # TIEMPOS DE ENTREGA
    mensaje += "📅 **TIEMPOS DE ENTREGA**\n"
    mensaje += "─" * 80 + "\n"
    if internos:
        lead_interno = max((int(p.get('lead_time', 0)) for p in internos if isinstance(p.get('lead_time'), (int, float))), default=1)
        almacen = internos[0].get('proveedor', 'Almacén')
        mensaje += f"   • Internos: {lead_interno} día{'s' if lead_interno != 1 else ''} ({almacen})\n"
    if externos:
        lead_externo = max((int(p.get('lead_time', 0)) for p in externos if isinstance(p.get('lead_time'), (int, float))), default=0)
        mensaje += f"   • Externos: {lead_externo} días\n"
    
    lead_max = max(
        max((int(p.get('lead_time', 0)) for p in internos if isinstance(p.get('lead_time'), (int, float))), default=0) if internos else 0,
        max((int(p.get('lead_time', 0)) for p in externos if isinstance(p.get('lead_time'), (int, float))), default=0) if externos else 0
    )
    mensaje += f"   • **Lead time máximo:** {lead_max} días\n\n"
    
    # PRÓXIMOS PASOS
    mensaje += "🔔 **PRÓXIMOS PASOS**\n"
    mensaje += "─" * 80 + "\n"
    paso = 1
    if internos:
        mensaje += f"{paso}. ✅ Reservar productos internos en almacén\n"
        paso += 1
    if externos:
        mensaje += f"{paso}. 📧 Enviar orden de compra a proveedor{'es' if len(externos_por_proveedor) > 1 else ''} externo{'s' if len(externos_por_proveedor) > 1 else ''}\n"
        paso += 1
    mensaje += f"{paso}. 📦 Coordinar logística de entrega\n"
    mensaje += f"{paso+1}. 💳 Procesar pago: ${total_general:.2f}\n\n"
    
    mensaje += "═" * 80 + "\n"
    mensaje += "✅ **Orden lista para procesar**\n"
    mensaje += "═" * 80
    
    
    return {
        "messages": [AIMessage(content=mensaje)]
    }


In [17]:
def process_internal_order(state: AgentState) -> AgentState:
    """
    Genera orden de retiro de inventario para productos INTERNOS.
    Notifica al almacén para preparación.
    """
    selecciones = state.get("selecciones_usuario", [])
    internos = [s for s in selecciones if s['tipo'] == 'INTERNO']
    
    if not internos:
        return {
            "messages": [AIMessage(content="⚠️ No hay productos internos para procesar")]
        }
    
    
    # Agrupar por almacén
    por_almacen = {}
    for prod in internos:
        almacen = prod.get('proveedor', 'Almacén General')
        if almacen not in por_almacen:
            por_almacen[almacen] = []
        por_almacen[almacen].append(prod)
    
    mensaje = "═" * 80 + "\n"
    mensaje += "📦 **ORDEN DE RETIRO DE INVENTARIO**\n"
    mensaje += "═" * 80 + "\n\n"
    
    total_general = 0
    
    for almacen, productos in por_almacen.items():
        mensaje += f"🏢 **ALMACÉN: {almacen}**\n"
        mensaje += "─" * 80 + "\n\n"
        
        subtotal_almacen = 0
        
        for i, prod in enumerate(productos, 1):
            precio = prod.get('precio', 0)
            stock = prod.get('stock', 'N/A')
            lead_time = prod.get('lead_time', 1)
            
            subtotal_almacen += precio
            total_general += precio
            
            mensaje += f"{i}. **{prod['codigo']}**\n"
            mensaje += f"   📝 Descripción: {prod['descripcion']}\n"
            mensaje += f"   📦 Cantidad a retirar: 1 unidad\n"
            mensaje += f"   💰 Valor: ${precio:.2f}\n"
            mensaje += f"   📊 Stock disponible: {stock} unidades\n"
            mensaje += f"   ⏱️  Tiempo de preparación: {lead_time} día{'s' if lead_time != 1 else ''}\n\n"
        
        mensaje += f"💰 **Subtotal {almacen}:** ${subtotal_almacen:.2f}\n"
        mensaje += "─" * 80 + "\n\n"
    
    # Resumen
    mensaje += "📋 **RESUMEN DE LA ORDEN**\n"
    mensaje += "─" * 80 + "\n"
    mensaje += f"   Total de productos: {len(internos)}\n"
    mensaje += f"   Almacenes involucrados: {len(por_almacen)}\n"
    mensaje += f"   Valor total: ${total_general:.2f}\n\n"
    
    # Próximos pasos
    mensaje += "🔔 **NOTIFICACIÓN AL ALMACÉN**\n"
    mensaje += "─" * 80 + "\n"
    for almacen in por_almacen.keys():
        mensaje += f"✅ Notificación enviada a: {almacen}\n"
    mensaje += "\n"
    
    mensaje += "📌 **INSTRUCCIONES:**\n"
    mensaje += "1. Preparar productos para retiro\n"
    mensaje += "2. Verificar estado y calidad\n"
    mensaje += "3. Embalar y etiquetar\n"
    mensaje += "4. Notificar cuando esté listo\n\n"
    
    mensaje += "═" * 80 + "\n"
    mensaje += "✅ **Orden de retiro generada exitosamente**\n"
    mensaje += "═" * 80
    
    
    return {
        "messages": [AIMessage(content=mensaje)]
    }


def generate_external_email(state: AgentState) -> AgentState:
    """
    Genera email automatizado para proveedores EXTERNOS.
    Formaliza el pedido de productos.
    """
    selecciones = state.get("selecciones_usuario", [])
    externos = [s for s in selecciones if s['tipo'] == 'EXTERNO']
    
    if not externos:
        return {
            "messages": [AIMessage(content="⚠️ No hay productos externos para procesar")]
        }
    
    
    # Agrupar por proveedor
    por_proveedor = {}
    for prod in externos:
        proveedor = prod.get('proveedor', 'Proveedor Desconocido')
        if proveedor not in por_proveedor:
            por_proveedor[proveedor] = []
        por_proveedor[proveedor].append(prod)
    
    mensaje = "═" * 80 + "\n"
    mensaje += "📧 **EMAILS GENERADOS PARA PROVEEDORES**\n"
    mensaje += "═" * 80 + "\n\n"
    
    for proveedor, productos in por_proveedor.items():
        mensaje += "─" * 80 + "\n"
        mensaje += f"**PARA: {proveedor}**\n"
        mensaje += "─" * 80 + "\n\n"
        
        # Formato de email
        mensaje += "```\n"
        mensaje += f"Para: {proveedor}\n"
        mensaje += "Asunto: Solicitud de Cotización y Pedido - Repuestos Industriales\n\n"
        mensaje += "Estimado proveedor,\n\n"
        mensaje += "Por medio de la presente, solicitamos cotización y disponibilidad para los siguientes repuestos:\n\n"
        
        total_proveedor = 0
        
        mensaje += "DETALLE DEL PEDIDO:\n"
        mensaje += "-" * 60 + "\n\n"
        
        for i, prod in enumerate(productos, 1):
            precio = prod.get('precio', 0)
            lead_time = prod.get('lead_time', 'N/A')
            
            total_proveedor += precio
            
            mensaje += f"{i}. Código: {prod['codigo']}\n"
            mensaje += f"   Descripción: {prod['descripcion']}\n"
            mensaje += f"   Cantidad solicitada: 1 unidad\n"
            mensaje += f"   Precio estimado: ${precio:.2f}\n"
            mensaje += f"   Lead time estimado: {lead_time} días\n\n"
        
        mensaje += "-" * 60 + "\n"
        mensaje += f"TOTAL ESTIMADO: ${total_proveedor:.2f}\n\n"
        
        mensaje += "INFORMACIÓN ADICIONAL:\n"
        mensaje += "- Forma de pago: A convenir\n"
        mensaje += "- Plazo de entrega: Según disponibilidad\n"
        mensaje += "- Dirección de entrega: A coordinar\n\n"
        
        mensaje += "Por favor, confirmen:\n"
        mensaje += "1. Disponibilidad de los productos\n"
        mensaje += "2. Precio final actualizado\n"
        mensaje += "3. Tiempo de entrega real\n"
        mensaje += "4. Condiciones comerciales\n\n"
        
        mensaje += "Quedamos a la espera de su pronta respuesta.\n\n"
        mensaje += "Saludos cordiales,\n"
        mensaje += "Departamento de Compras\n"
        mensaje += "```\n\n"
        
        mensaje += f"✅ Email preparado para: **{proveedor}**\n"
        mensaje += f"📊 Productos solicitados: {len(productos)}\n"
        mensaje += f"💰 Total estimado: ${total_proveedor:.2f}\n\n"
    
    mensaje += "═" * 80 + "\n"
    mensaje += f"📧 **{len(por_proveedor)} email(s) generado(s) exitosamente**\n"
    mensaje += "═" * 80 + "\n\n"
    
    mensaje += "🔔 **PRÓXIMOS PASOS:**\n"
    mensaje += "1. Revisar y enviar emails a proveedores\n"
    mensaje += "2. Esperar confirmación de disponibilidad\n"
    mensaje += "3. Negociar condiciones comerciales\n"
    mensaje += "4. Formalizar pedido\n"
    mensaje += "5. Coordinar logística de entrega\n"
        
    return {
        "messages": [AIMessage(content=mensaje)]
    }


def route_by_product_type(state: AgentState) -> str:
    """
    Decide el flujo según el tipo de productos seleccionados.
    - Solo INTERNOS → process_internal_order
    - Solo EXTERNOS → generate_external_email
    - MIXTOS → process_both (ambos procesos)
    """
    selecciones = state.get("selecciones_usuario", [])
    
    if not selecciones:
        return "end"
    
    internos = [s for s in selecciones if s['tipo'] == 'INTERNO']
    externos = [s for s in selecciones if s['tipo'] == 'EXTERNO']
    
    tiene_internos = len(internos) > 0
    tiene_externos = len(externos) > 0
    
    if tiene_internos and tiene_externos:
        return "both"  # Tiene ambos tipos
    elif tiene_internos:
        return "internal_only"  # Solo internos
    elif tiene_externos:
        return "external_only"  # Solo externos
    else:
        return "end"


def process_both_types(state: AgentState) -> AgentState:
    """
    Procesa tanto productos INTERNOS como EXTERNOS.
    Genera orden interna Y emails externos.
    """
    # Procesar internos
    result_interno = process_internal_order(state)
    mensaje_interno = result_interno["messages"][0].content
    
    # Procesar externos
    result_externo = generate_external_email(state)
    mensaje_externo = result_externo["messages"][0].content
    
    # Combinar mensajes
    mensaje_completo = mensaje_interno + "\n\n" + mensaje_externo
    
    return {
        "messages": [AIMessage(content=mensaje_completo)]
    }


In [18]:
def semantic_search_internal(state: AgentState) -> AgentState:
    """
    Búsqueda semántica SOLO en inventario INTERNO.
    Verifica qué productos fueron encontrados y cuáles no.
    Combina búsqueda semántica + verificación de stock en un solo paso.
    """
    from sentence_transformers import SentenceTransformer
    
    product_requests = state.get("product_requests", [])
    
    
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    todos_repuestos = []
    codigos_unicos_global = set()
    codigos_encontrados_global = []
    productos_sin_resultados = []
    resultados_por_codigo = {}  # Para el reranking
    mensaje_productos = ""
    
    for idx, product_req in enumerate(product_requests, 1):
        product_query = product_req.get("name", "")
        
        if not product_query:
            continue
                
        query_embedding = model.encode(product_query).tolist()
        
        # Pipeline con FILTRO INTERNO
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index_repuestos",
                    "path": "embedding_vector",
                    "queryVector": query_embedding,
                    "numCandidates": 100,
                    "limit": 5
                }
            },
            {
                "$project": {
                    "id_repuesto": 1,
                    "repuesto_descripcion": 1,
                    "categoria": 1,
                    "marca": 1,
                    "modelo": 1,
                    "proveedor_tipo": 1,
                    "proveedor_nombre": 1,
                    "stock_disponible": 1,
                    "costo_unitario": 1,
                    "lead_time_dias": 1,
                    "score": {"$meta": "vectorSearchScore"}
                }
            }
        ]
        
        try:
            resultados_raw = list(collection.aggregate(pipeline))
            
            # Filtrar por proveedor_tipo DESPUÉS de la búsqueda
            resultados = [r for r in resultados_raw if r.get('proveedor_tipo') == 'INTERNAL']
            
            if not resultados or len(resultados) == 0:
                print(f"   ❌ No encontrado en inventario interno\n")
                productos_sin_resultados.append({
                    "idx": idx,
                    "name": product_query,
                    "product_req": product_req
                })
                mensaje_productos += f"**{idx}. {product_query}**\n   ❌ No disponible internamente\n\n"
                continue
            
            # Filtrar por stock disponible y score
            resultados_con_stock = [
                r for r in resultados 
                if r.get('score', 0) >= 0.50 and r.get('stock_disponible', 0) > 0
            ]
            
            if not resultados_con_stock:
                print(f"   ⚠️  Encontrado pero sin stock disponible\n")
                productos_sin_resultados.append({
                    "idx": idx,
                    "name": product_query,
                    "product_req": product_req
                })
                mensaje_productos += f"**{idx}. {product_query}**\n   ⚠️  Sin stock interno\n\n"
                continue
            
            
            mensaje_productos += f"**{idx}. {product_query}**\n"
            
            for i, r in enumerate(resultados_con_stock[:3], 1):  # Top 3
                if '_id' in r:
                    r['_id'] = str(r['_id'])
                
                repuesto = Repuesto(
                    id_repuesto=r['id_repuesto'],
                    repuesto_descripcion=r['repuesto_descripcion'],
                    marca=r.get('marca', 'N/A'),
                    modelo=r.get('modelo', 'N/A'),
                    categoria=r.get('categoria', 'N/A'),
                    score=r.get('score', 0)
                )
                
                todos_repuestos.append(repuesto)
                
                if repuesto.id_repuesto not in codigos_unicos_global:
                    codigos_encontrados_global.append(repuesto.id_repuesto)
                    codigos_unicos_global.add(repuesto.id_repuesto)
                
                # Guardar en formato para reranking
                codigo = repuesto.id_repuesto
                if codigo not in resultados_por_codigo:
                    resultados_por_codigo[codigo] = []
                resultados_por_codigo[codigo].append(r)
                
                stock = r.get('stock_disponible', 0)
                proveedor = r.get('proveedor_nombre', 'N/A')                
                mensaje_productos += f"   {i}. **{repuesto.id_repuesto} ({repuesto.marca})** - Stock: {stock}\n"
            
            mensaje_productos += "\n"
                    
        except Exception as e:
            print(f"   ❌ Error: {e}\n")
            productos_sin_resultados.append({
                "idx": idx,
                "name": product_query,
                "product_req": product_req
            })
    
    # Preparar mensaje
    mensaje = "🔍 **Resultados de búsqueda interna:**\n\n"
    mensaje += mensaje_productos
    
    if productos_sin_resultados:
        mensaje += f"────────────────────────────────\n"
        mensaje += f"⚠️  **{len(productos_sin_resultados)} producto(s) no disponible(s) internamente**\n"
        mensaje += f"🌐 Buscando en proveedores externos..."
    else:
        mensaje += "✅ Todos los productos disponibles internamente"
    
    return {
        "messages": [AIMessage(content=mensaje)],
        "codigos_repuestos": codigos_encontrados_global,
        "repuestos_encontrados": todos_repuestos,
        "productos_sin_match_interno": productos_sin_resultados,
        "resultados_internos": resultados_por_codigo,  # Para el reranking
        "info_completa": True
    }

def semantic_search_external(state: AgentState) -> AgentState:
    """
    Búsqueda semántica SOLO en proveedores EXTERNOS.
    Solo busca productos que NO se encontraron internamente.
    """
    from sentence_transformers import SentenceTransformer
    
    productos_sin_match = state.get("productos_sin_match_interno", [])
    
    if not productos_sin_match:
        print("✅ Todos los productos se encontraron internamente\n")
        return {}
    
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    repuestos_externos = list(state.get("repuestos_encontrados", []))
    codigos_existentes = set(state.get("codigos_repuestos", []))
    codigos_externos = []
    resultados_externos_por_codigo = {}  # Para el reranking
    mensaje_externos = ""
    
    for item in productos_sin_match:
        product_query = item.get("name", "")
        idx = item.get("idx", "")
                
        query_embedding = model.encode(product_query).tolist()
        
        # Pipeline con FILTRO EXTERNO
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index_repuestos",
                    "path": "embedding_vector",
                    "queryVector": query_embedding,
                    "numCandidates": 100,
                    "limit": 5
                }
            },
            {
                "$project": {
                    "id_repuesto": 1,
                    "repuesto_descripcion": 1,
                    "categoria": 1,
                    "marca": 1,
                    "modelo": 1,
                    "proveedor_tipo": 1,
                    "proveedor_nombre": 1,
                    "costo_unitario": 1,
                    "lead_time_dias": 1,
                    "score": {"$meta": "vectorSearchScore"}
                }
            }
        ]
        
        try:
            resultados_raw = list(collection.aggregate(pipeline))
            
            # Filtrar por proveedor_tipo DESPUÉS de la búsqueda
            resultados = [r for r in resultados_raw if r.get('proveedor_tipo') == 'EXTERNAL']
            
            if not resultados:
                print(f"   ❌ No encontrado en externos\n")
                mensaje_externos += f"**{idx}. {product_query}**\n   ❌ No disponible\n\n"
                continue
            
            # Filtrar por score
            resultados_validos = [r for r in resultados if r.get('score', 0) >= 0.50]
                        
            mensaje_externos += f"**{idx}. {product_query}**\n"
            
            for i, r in enumerate(resultados_validos[:3], 1):  # Top 3
                if '_id' in r:
                    r['_id'] = str(r['_id'])
                
                repuesto = Repuesto(
                    id_repuesto=r['id_repuesto'],
                    repuesto_descripcion=r['repuesto_descripcion'],
                    marca=r.get('marca', 'N/A'),
                    modelo=r.get('modelo', 'N/A'),
                    categoria=r.get('categoria', 'N/A'),
                    score=r.get('score', 0)
                )
                
                repuestos_externos.append(repuesto)
                
                if repuesto.id_repuesto not in codigos_existentes:
                    codigos_externos.append(repuesto.id_repuesto)
                    codigos_existentes.add(repuesto.id_repuesto)
                
                # Guardar en formato para reranking
                codigo = repuesto.id_repuesto
                if codigo not in resultados_externos_por_codigo:
                    resultados_externos_por_codigo[codigo] = []
                resultados_externos_por_codigo[codigo].append(r)
                
                proveedor = r.get('proveedor_nombre', 'N/A')
                lead_time = r.get('lead_time_dias', 'N/A')                
                mensaje_externos += f"   {i}. **{repuesto.id_repuesto} ({repuesto.marca})** - {proveedor}\n"
            
            mensaje_externos += "\n"
                    
        except Exception as e:
            print(f"   ❌ Error: {e}\n")
    
    # Combinar códigos
    todos_codigos = list(state.get("codigos_repuestos", [])) + codigos_externos
    
    mensaje = "🌐 **Búsqueda en proveedores externos:**\n\n"
    mensaje += mensaje_externos
    mensaje += f"────────────────────────────────\n"
    mensaje += f"✅ {len(codigos_externos)} producto(s) adicional(es) encontrado(s) externamente"
    
    return {
        "messages": [AIMessage(content=mensaje)],
        "codigos_repuestos": todos_codigos,
        "repuestos_encontrados": repuestos_externos,
        "resultados_externos": resultados_externos_por_codigo  # Para el reranking
    }

def need_external_search(state: AgentState) -> str:
    """
    Decide si se necesita búsqueda externa.
    """
    productos_sin_match = state.get("productos_sin_match_interno", [])
    
    if productos_sin_match and len(productos_sin_match) > 0:
        return "search_external"
    else:
        return "reranking"


## 8. Generación del Grafo

In [19]:
memory = MemorySaver()

# Definimos el grafo
graph_builder = StateGraph(AgentState)

# Agrego los nodos al grafo
graph_builder.add_node("validation", classify_request)
graph_builder.add_node("set_val_message", set_val_message)
# Nodos para extracción y verificación
graph_builder.add_node("extract_products_info", extract_products_info) 
graph_builder.add_node("check_product_info_completeness", check_product_info_completeness) 
# Nodos de búsqueda semántica
graph_builder.add_node("semantic_search_internal", semantic_search_internal)
graph_builder.add_node("semantic_search_external", semantic_search_external)
# Nodo de ranking
graph_builder.add_node("reranking", generate_ranking)

# Conecto los nodos
graph_builder.add_edge(START, "validation")

# Desde validation: si es pedido de repuestos → extract_products_info
graph_builder.add_conditional_edges(
    "validation",
    route_classification,
    {
        "continue": "extract_products_info",
        "end": "set_val_message"
    }
)

graph_builder.add_edge("set_val_message", END)

# Desde extracción → verificar
graph_builder.add_edge("extract_products_info", "check_product_info_completeness")

# Desde verificación: si info completa → semantic_search_internal, si no → END
graph_builder.add_conditional_edges(
    "check_product_info_completeness",
    route_after_extraction_check,
    {
        "semantic_search": "semantic_search_internal",  # Ir directo a búsqueda interna
        "request_more_info": END
    }
)

# Desde semantic_search_internal: decidir si buscar externamente o ir a ranking
graph_builder.add_conditional_edges(
    "semantic_search_internal",
    need_external_search,
    {
        "search_external": "semantic_search_external",
        "reranking": "reranking"
    }
)

# Desde semantic_search_external → reranking
graph_builder.add_edge("semantic_search_external", "reranking")

# Desde reranking → END
# Nodo de Human in the Loop
graph_builder.add_node("human_in_the_loop", human_in_the_loop_selection)
graph_builder.add_node("process_selection", process_user_selection)
graph_builder.add_node("process_order", process_order)
graph_builder.add_node("process_internal_order", process_internal_order)
graph_builder.add_node("generate_external_email", generate_external_email)
graph_builder.add_node("process_both_types", process_both_types)

# Conexiones actualizadas
graph_builder.add_edge("reranking", "human_in_the_loop")

# Aquí usamos interrupt_before para detener la ejecución y esperar input del usuario
# El grafo se detendrá ANTES de process_selection para que el usuario responda
graph_builder.add_edge("human_in_the_loop", "process_selection")

graph_builder.add_conditional_edges(
    "process_selection",
    route_after_selection,
    {
        "process_order": "process_order",  # DEPRECATED: solo para mostrar resumen
        "end": END  # Usuario canceló
    }
)

# Routing condicional según tipo de productos
graph_builder.add_conditional_edges(
    "process_order",
    route_by_product_type,
    {
        "internal_only": "process_internal_order",  # Solo productos internos
        "external_only": "generate_external_email",  # Solo productos externos
        "both": "process_both_types",  # Ambos tipos
        "end": END
    }
)

# Después de cada tipo de procesamiento, terminar
graph_builder.add_edge("process_internal_order", END)
graph_builder.add_edge("generate_external_email", END)
graph_builder.add_edge("process_both_types", END)

# Compilar el grafo
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_after=["human_in_the_loop"]  # Pausa DESPUÉS de presentar opciones al usuario
)


## 9. Inicialización del agente

In [20]:
def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    
    # Estado inicial con todos los campos
    estado_inicial = {
        "messages": [HumanMessage(mensaje_usuario)],
        "validation_result": None,
        "conversation_result": None,
        "product_description": [],
        "product_requests": [],
        "codigos_repuestos": None,
        "repuestos_encontrados": None,
        "productos_sin_match_interno": None,
        "selecciones_usuario": None,
        "repuestos_seleccionados": False,
        "info_completa": False,
        "optimized_query": None,
        "semantic_results": None,
        "resultados_internos": {},
        "resultados_externos": {},
        "disponibilidad": None,
        "codigos_para_externos": None,
        "recomendaciones_llm": None
    }
    
    result = graph.invoke(estado_inicial, config)
    
    # Loop de conversación
    while True:
        # Mostrar solo NUEVOS mensajes del agente (AIMessage)
        mensajes_actuales = result.get("messages", [])
        
        # Buscar el último AIMessage (mensaje del agente)
        ultimo_mensaje_agente = None
        for msg in reversed(mensajes_actuales):
            if isinstance(msg, AIMessage):
                ultimo_mensaje_agente = msg.content
                break
        
        if ultimo_mensaje_agente:
            print(f"\n🤖 Agente: {ultimo_mensaje_agente}")
        
        # Verificar si hay un interrupt (Human in the Loop)
        snapshot = graph.get_state(config)
        proximos_nodos = snapshot.next if hasattr(snapshot, 'next') else []
        
        # Si el grafo está pausado después de human_in_the_loop
        if proximos_nodos and "process_selection" in proximos_nodos:
            # Loop hasta obtener una selección válida
            while True:
                print(f"\n{'─'*60}")
                nuevo_mensaje = input("\n👤 Tu selección: ").strip()
                
                if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
                    print("\n👋 Conversación terminada")
                    return result
                
                # Actualizar el estado sin reiniciar
                graph.update_state(config, {
                    "messages": [HumanMessage(content=nuevo_mensaje)]
                })
                
                # Continuar desde donde se pausó (None = continuar, no reiniciar)
                result = graph.invoke(None, config)
                
                # Verificar si el usuario seleccionó productos válidos
                repuestos_seleccionados = result.get("repuestos_seleccionados", False)
                
                if repuestos_seleccionados:
                    # Selección exitosa o cancelación, salir del loop
                    break
                else:
                    # Respuesta inválida, mostrar el mensaje del AGENTE y re-preguntar
                    mensajes_actuales = result.get("messages", [])
                    # Filtrar solo AIMessages (mensajes del agente)
                    for msg in reversed(mensajes_actuales):
                        if isinstance(msg, AIMessage):
                            print(f"\n🤖 Agente: {msg.content}")
                            break
                    # Continuar el loop para pedir nueva entrada
            
            # Después de selección válida, continuar con el flujo
            continue
        
        # Verificar si se completó el pedido (hay selecciones)
        selecciones = result.get("selecciones_usuario")
        if selecciones is not None:
            if len(selecciones) > 0:
                print(f"\n{'='*60}")
                print("✅ Pedido procesado exitosamente")
                print(f"   Productos seleccionados: {len(selecciones)}")
                
                # Mostrar resumen
                internos = [s for s in selecciones if s['tipo'] == 'INTERNO']
                externos = [s for s in selecciones if s['tipo'] == 'EXTERNO']
                
                if internos:
                    print(f"   • Internos: {len(internos)}")
                if externos:
                    print(f"   • Externos: {len(externos)}")
                
                print(f"{'='*60}\n")
            else:
                print("\n❌ Pedido cancelado por el usuario\n")
            break
        
        # Si no hay información completa, pedir más detalles
        if result.get("info_completa") == False:
            print(f"\n{'─'*60}")
            nuevo_mensaje = input("\n👤 Tú: ").strip()
            
            if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
                print("\n👋 Conversación terminada")
                break
            
            # Para el caso de pedir más info, sí necesitamos pasar el mensaje
            result = graph.invoke({
                "messages": [HumanMessage(content=nuevo_mensaje)]
            }, config)
            continue
        
        # Si llegamos aquí sin break, algo salió mal o terminó el flujo
        break
    
    return result


## 10. Inicio de Conversación

### Ejemplos de uso del agente

**Ejemplos de códigos de repuestos en la base de datos:**
- R-0001: Rodamiento rígido de bolas 6204 2RS
- R-0002: Filtro de aceite motor diésel
- R-0005: Bomba centrífuga 3 HP
- R-0010: Manómetro glicerina 0-16 bar

**Categorías disponibles:**
- RODAMIENTO, FILTRO, CORREA, SENSOR, BOMBA, ELECTRICO, NEUMATICA, MECANICO, INSTRUMENTO


In [ ]:
#main
print("="*60)
print("🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS")
print("="*60)
print("\nBienvenido al sistema de búsqueda de repuestos.")
print("El agente te ayudará a encontrar el repuesto que necesitas.")
print("\nPuedes escribir 'salir' en cualquier momento para terminar.\n")
print("-"*60)

#mensaje_usuario = input("\n👤 Tú: ")

#Para debugging

##Dos productos
#mensaje_usuario = "Necesito un Rodamiento rígido de bolas modelo 6204 2RS y dos Kits reparación válvula"

##Producto solo externo
mensaje_usuario = "Cartucho filtro de aire compresor 250 m3/h y dos Kits reparación válvula"

resultado = iniciar_agente(mensaje_usuario)

#Repuesto con el codigo R-0001
#Repuesto con codigo R-0005 y R-0002


🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS

Bienvenido al sistema de búsqueda de repuestos.
El agente te ayudará a encontrar el repuesto que necesitas.

Puedes escribir 'salir' en cualquier momento para terminar.

------------------------------------------------------------
✅ Producto 1: 'Cartucho filtro de aire compresor 250 m3/h'
   → Información completa: La descripción indica claramente el tipo de repuesto (filtro de aire), cumpliendo el requisito mínimo para una búsqueda, aunque no incluye marca ni modelo, que son opcionales.

✅ Producto 2: 'dos Kits reparación válvula'
   → Información completa: La descripción indica claramente el tipo de repuesto como 'Kit de reparación de válvula', cumpliendo el requisito mínimo de identificar el tipo. Falta la marca y el número de parte, que son opcionales pero recomendados.

Estas son las opciones!! ═══════════════════════════════════════════════════════════════
REPUESTO: R-0002
═══════════════════════════════════════════════════════════════
Descripc